참고 링크: https://github.com/TeamOTK/Search/blob/main/src/Search.py

In [1]:
from pypdf import PdfReader
from openai import OpenAI
import os
from dotenv import load_dotenv
import numpy as np

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

PDF에서 텍스트를 읽어와서 청크로 분할

In [2]:
reader = PdfReader("data/spiderman1.pdf")
chunks = []
chunk_length = 1000

for page in reader.pages:
    text_page = page.extract_text()
    chunks.extend([text_page[i: i+chunk_length] for i in range(0, len(text_page), chunk_length)])

크로마디비에 벡터 저장

In [10]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_texts(chunks, embeddings)

In [11]:
retriever = vectorstore.as_retriever()

In [12]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="input",
    return_messages=True,
    output_key="output",
)

In [13]:
from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model_name="gpt-4-1106-preview",
    temperature=0
)

In [14]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    verbose=True,
    return_source_documents=True
)

In [15]:
from langchain.agents import Tool

tools = [
    Tool(
        name="doc_search_tool",
        func=qa,
        description=(
            "This tool is used to retrieve information from the knowledge base"
        )
    )
]

In [17]:
system_message = """
    You always follow these guidelines:

        -If the answer isn't available within the context, state that fact
        -Otherwise, answer to your best capability, refering to source of documents provided
        -Only use examples if explicitly requested
        -Do not introduce examples outside of the context
        -Do not answer if context is absent
        -Limit responses to three or four sentences for clarity and conciseness
        -You must answer in Koreans
"""

In [18]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    memory=memory,
    return_source_documents=True,
    return_intermediated_steps=True,
    agent_kwargs={"system_message": system_message}
)

In [19]:
query = "스파이더맨의 주인공은 누구야"
result = agent(query)
print(result["output"])



> Entering new RetrievalQA chain...

> Finished chain.
어메이징 스파이더맨 실사영화 시리즈의 주인공은 피터 파커이며, 이 역할을 맡은 배우는 앤드류 가필드입니다.
